In [1]:
import ast
import re
import pandas as pd
import numpy as np
from collections import defaultdict, Counter

In [2]:
df = pd.read_csv('povarenok_recipes_2021_06_16.csv') # all text in df is russian
df.head()

,url,name,ingredients
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах..."
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку..."
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '..."
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':..."
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146582 entries, 0 to 146581
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   url          146582 non-null  object
 1   name         146582 non-null  object
 2   ingredients  146581 non-null  object
dtypes: object(3)
memory usage: 3.4+ MB


In [5]:
type(df["ingredients"][0])

str

In [ ]:
# change str to dict in df['ingredients']
def to_dict(x):
    try:
        return ast.literal_eval(x) if isinstance(x, str) else {}
    except:
        return {}

df['ingredients'] = df['ingredients'].apply(to_dict)

In [ ]:
def clean_ingredient(ing):
  result = {}
  if isinstance(ing, dict):
    for k, v in ing.items():
      k = k.strip().lower()
      v = str(v).strip().lower()

      match = re.match(r'([\d.,]+)?\s*([а-яa-z]*)', v)   # devide the values by name, quantity and unit
      qty = match.group(1)
      unit = match.group(2)
      result[k] = {'quantity': qty, 'unit': unit}  
  return result

df['ingredients'] = df['ingredients'].apply(clean_ingredient)
df.head()

,url,name,ingredients
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'молоко': {'quantity': '250', 'unit': 'мл'}, ..."
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'сыр твердый': {'quantity': None, 'unit': 'no..."
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'баклажан': {'quantity': '3', 'unit': 'шт'}, ..."
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'фарш куриный': {'quantity': '800', 'unit': '..."
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'вишня': {'quantity': '1', 'unit': 'кг'}, 'во..."


In [ ]:
df['ingredients'].iloc[0]

{'молоко': {'quantity': '250', 'unit': 'мл'},
 'клубника': {'quantity': '200', 'unit': 'г'},
 'сахар': {'quantity': '15', 'unit': 'г'}}

In [ ]:
# i need average quantity and unit 

ingredient_values = defaultdict(list)
ingredient_units = defaultdict(list)

for ing_dict in df['ingredients']:
  for name, info in ing_dict.items():
    qty = info.get('quantity')
    unit = info.get('unit')

    if qty not in [None, 'none', np.nan, 'None']:
      try:
        qty_val = float(str(qty).replace(',','.'))
        ingredient_values[name].append(qty_val)
      except:
        pass
    if unit not in [None, 'none', np.nan, 'None']:
      ingredient_units[name].append(unit)

ingredient_means = {k: np.mean(v) for k, v in ingredient_values.items() if v}
ingredient_mode_unit = {k: Counter(v).most_common(1)[0][0] for k, v in ingredient_units.items() if v}

In [ ]:
# and  now i change nan to average values

def fill_nan_quantities_and_units(ing_dict):
  new_dict = {}
  for name, info in ing_dict.items():
    qty = info.get('quantity')
    unit = info.get('unit')

    if qty not in [None, 'none', np.nan, 'None']:
      new_qty = round(ingredient_means.get(name, 1.0), 2)
    else:
      try:
        new_qty = float(str(qty).replace(',','.'))
      except:
        new_qty = 1.0

    if unit not in [None, 'none', np.nan, 'None']:
      new_unit = unit
    else:
      new_unit = ingredient_mode_unit.get(name, 'шт')

    new_dict[name] = {'quantity': new_qty, 'unit': new_unit}
  return new_dict

df['ingredients'] = df['ingredients'].apply(fill_nan_quantities_and_units)

In [ ]:
# example, take 14 random meals 
week_meals = df.sample(14, random_state=42)

In [ ]:
# sum values and genereted shopping list

def aggregate_ingredients(selected_meals):
    shopping_list = defaultdict(lambda: {'quantity': 0.0, 'unit': None})

    for ing_dict in selected_meals['ingredients']:
        for name, info in ing_dict.items():
            qty = info.get('quantity')
            unit = info.get('unit')

            try:
                qty_val = float(str(qty).replace(',', '.')) if qty else 1.0
            except:
                qty_val = 1.0

            if shopping_list[name]['unit'] in [unit, None]:
                if not isinstance(shopping_list[name]['quantity'], (int, float)):
                    shopping_list[name]['quantity'] = 0.0

                shopping_list[name]['quantity'] += qty_val
                shopping_list[name]['unit'] = unit
            else:
                existing_qty = str(shopping_list[name]['quantity'])
                shopping_list[name]['quantity'] = f"{existing_qty} + {qty} {unit}"

    return shopping_list

In [ ]:
def show_recipes(df, dish_names):

    for dish in dish_names:
        row = df.loc[df['name'] == dish]

        if row.empty:
            print(f"Блюдо '{dish}' не найдено.\n")
            continue

        ingredients = row.iloc[0]['ingredients']

        if isinstance(ingredients, str):
            try:
                ingredients = ast.literal_eval(ingredients)
            except Exception as e:
                print(f"Не удалось распарсить ингредиенты для '{dish}': {e}")
                continue

        print(f"{dish}\nИнгредиенты:")
        for name, info in ingredients.items():
            quantity = info.get('quantity', '')
            unit = info.get('unit', '')
            print(f" - {name}: {quantity} {unit}")
        print("\n" + "-" * 40 + "\n")

In [21]:
def test(df):
    week_meals = df.sample(14)
    shopping_list = aggregate_ingredients(week_meals)
    print(f"Meals for a week\n")

    shopping_list = aggregate_ingredients(week_meals)
    dish_name_list = [k for k in week_meals['name']]
    show_recipes(week_meals, dish_name_list)
    
    print(f"\n\nIngredients\n")

    n = 1
    for k, v in shopping_list.items():
        print(f"{n}. {k}: {v['quantity']} {v['unit'] or ''}")
        n += 1

test(df)

Meals for a week

Куриное филе с сыром дорблю и беконом
Ингредиенты:
 - бедро куриное: 104.14 шт
 - сыр голубой: 84.3 г
 - соевый соус: 11.54 ст
 - соус: 1.0 ст
 - чеснок: 4.74 зуб
 - овощи: 1.0 г
 - зелень: 1.0 пуч
 - бекон: 1.0 г
 - соль: 1.0 по

----------------------------------------

Соленые огурцы с красной смородиной на зиму
Ингредиенты:
 - огурец: 21.9 г
 - смородина красная: 126.51 г
 - перец черный: 2.57 шт
 - чеснок: 4.74 зуб
 - укроп: 5.66 шт
 - соль: 4.11 ст
 - сахар: 61.59 ч

----------------------------------------

Щи рахмановские
Ингредиенты:
 - рыба: 1.0 г
 - щавель: 1.0 пуч
 - лук репчатый: 13.66 шт
 - морковь: 29.85 шт
 - картофель: 82.89 шт
 - вода: 129.49 л
 - сметана: 1.0 г
 - соль: 1.0 по
 - перец душистый: 1.0 шт
 - лист лавровый: 1.0 шт

----------------------------------------

Главная  > 
  Рецепты
        > 
    
версия для печати
Ингредиенты:
 - мука пшеничная: 141.88 г
 - масло сливочное: 92.05 г
 - сахар: 61.59 г
 - яйцо куриное: 2.86 шт
 - соль: 4.11 щ